In [1]:
import pandas as pd

In [2]:
file = 'Задание_для_соискателя.xlsx'

zlog = 'zlog'
params = 'params'

In [3]:
# читаем файлы

df_zlog = pd.read_excel(file, sheet_name=zlog, skiprows=5)
df_params = pd.read_excel(file, sheet_name=params, skiprows=1)

In [4]:
# смотрим

display(df_zlog.head())
display(df_params.head())

,zid,action,time,bank
0,1,z-created,2020-12-09 16:13:45,0
1,1,stto-ac,2020-12-09 16:13:45,0
2,1,edited,2020-12-09 16:15:46,0
3,1,stto-cs,2020-12-09 16:16:00,0
4,2,z-created,2020-12-09 16:55:04,0


,zid,sumbg
0,1,618799.50
1,2,15334644.60
2,3,6415667.37
3,4,9654890.52
4,5,1587607.42


# 1

In [5]:
# соединяем
df = df_zlog.merge(df_params, on='zid', how='left')

In [6]:
df

,zid,action,time,bank,sumbg
0,1,z-created,2020-12-09 16:13:45,0,618799.50
1,1,stto-ac,2020-12-09 16:13:45,0,618799.50
2,1,edited,2020-12-09 16:15:46,0,618799.50
3,1,stto-cs,2020-12-09 16:16:00,0,618799.50
4,2,z-created,2020-12-09 16:55:04,0,15334644.60
...,...,...,...,...,...
18749,103,stto-cr,2021-03-04 13:33:25,bank5,4481896.64
18750,103,stto-ar,2021-03-04 13:33:32,bank5,4481896.64
18751,850,stto-pa,2021-03-11 03:51:35,bank5,139657.10
18752,857,stto-pa,2021-03-21 01:11:06,bank5,72602.40


In [7]:
# необходимые события

cols = [
    'stto-bgs', # дата выдачи
    'stto-bms'  # дата одобрения
]

# фильтруем данные по событиям
df_filtered = df[df['action'].isin(cols)].reset_index(drop=True)

# сортируем по времени, далее для каждого договора и события создаем нумерацию по порядку
df_filtered['row_num'] = df_filtered.sort_values(by='time').groupby(['zid', 'action'])['bank'].cumcount() + 1

In [8]:
# пример
df_filtered[df_filtered['zid'] == 377]

,zid,action,time,bank,sumbg,row_num
859,377,stto-bms,2020-12-16 12:00:43,bank5,22190813.04,1
927,377,stto-bms,2020-12-17 10:00:14,bank5,22190813.04,2
948,377,stto-bms,2020-12-17 12:05:33,bank5,22190813.04,3
1063,377,stto-bgs,2020-12-18 14:02:47,bank5,22190813.04,1
1654,377,stto-bgs,2021-02-08 11:12:27,bank5,22190813.04,2


In [9]:
# выкидываем все события кроме первого
df_clean = df_filtered[df_filtered['row_num'] == 1].reset_index(drop=True)

# выкидываем нумерацию
df_clean = df_clean[['zid', 'action', 'time', 'bank', 'sumbg']]

In [10]:
df_clean

,zid,action,time,bank,sumbg
0,23,stto-bms,2020-12-10 11:19:46,bank1,7096956.68
1,38,stto-bms,2020-12-10 13:00:15,bank1,783540.06
2,24,stto-bms,2020-12-10 13:49:56,bank1,7356104.60
3,34,stto-bms,2020-12-10 13:52:44,bank1,3374827.74
4,35,stto-bms,2020-12-10 14:22:36,bank1,11873974.02
...,...,...,...,...,...
993,828,stto-bgs,2021-01-15 15:59:59,bank5,20295738.04
994,853,stto-bgs,2021-01-21 17:47:32,bank5,137543.45
995,243,stto-bgs,2021-01-25 15:49:03,bank5,3694806.50
996,847,stto-bgs,2021-01-28 17:44:57,bank5,446964.00


In [12]:
# даты одобрения
df_odobr = df_clean[df_clean['action'] == 'stto-bms'][['zid', 'time', 'bank']].rename(columns={'time': 'data_odobr'})

# даты выдачи
df_vidacha = df_clean[df_clean['action'] == 'stto-bgs'][['zid', 'time', 'sumbg']].rename(columns={'time': 'data_vidachi'})

# соединяем
df_final = df_odobr.merge(df_vidacha, on='zid', how='outer')

In [13]:
df_final

,zid,data_odobr,bank,data_vidachi,sumbg
0,23,2020-12-10 11:19:46,bank1,NaT,NaN
1,38,2020-12-10 13:00:15,bank1,NaT,NaN
2,24,2020-12-10 13:49:56,bank1,NaT,NaN
3,34,2020-12-10 13:52:44,bank1,NaT,NaN
4,35,2020-12-10 14:22:36,bank1,NaT,NaN
...,...,...,...,...,...
973,838,2020-12-25 14:00:09,bank5,NaT,NaN
974,865,2020-12-25 17:55:22,bank5,NaT,NaN
975,879,2020-12-28 12:58:01,bank5,2021-01-11 12:35:01,9836665.08
976,442,2020-12-28 16:55:14,bank5,NaT,NaN


In [14]:
# считаем № корзины по условию

df_final['bucket'] = df_final['sumbg'].apply(
    lambda x: (x < 500000 and 1) or 
              (x <= 1000000 and 2) or 
              (x <= 5000000 and 3) or 
              (x > 5000000 and 4)
)

In [15]:
#  выкидываем лишние колонки и сортируем по порядку из текста задачи
df_final = df_final[['zid', 'bank', 'sumbg', 'data_odobr', 'data_vidachi', 'bucket']]

In [16]:
df_final

,zid,bank,sumbg,data_odobr,data_vidachi,bucket
0,23,bank1,NaN,2020-12-10 11:19:46,NaT,False
1,38,bank1,NaN,2020-12-10 13:00:15,NaT,False
2,24,bank1,NaN,2020-12-10 13:49:56,NaT,False
3,34,bank1,NaN,2020-12-10 13:52:44,NaT,False
4,35,bank1,NaN,2020-12-10 14:22:36,NaT,False
...,...,...,...,...,...,...
973,838,bank5,NaN,2020-12-25 14:00:09,NaT,False
974,865,bank5,NaN,2020-12-25 17:55:22,NaT,False
975,879,bank5,9836665.08,2020-12-28 12:58:01,2021-01-11 12:35:01,4
976,442,bank5,NaN,2020-12-28 16:55:14,NaT,False


# 2